# **Explore prompt engineering techniques to optimize LLM responses for different applications.**

## Overview
This notebook demonstrates various **prompt engineering techniques** to optimize Large Language Model (LLM) responses for different applications. We'll explore 5 key techniques using Microsoft's Phi-3 model:

### Learning Objectives:
1. **Understand prompt engineering fundamentals** - How different prompting strategies affect model output
2. **Implement role assignment** - Guide the model to adopt specific personas or expertise levels
3. **Apply chain-of-thought prompting** - Break down complex reasoning into step-by-step processes
4. **Use output constraints** - Control response format, length, and structure
5. **Leverage few-shot learning** - Provide examples to guide model behavior

### Techniques Covered:
- **Role Assignment** - Making the AI adopt specific personas
- **Chain-of-Thought** - Step-by-step reasoning
- **Output Constraints** - Controlling response format
- **Hypothetical Scenarios** - Context-based prompting
- **Few-Shot Learning** - Learning from examples

---

## Environment Setup

Before we begin exploring prompt engineering techniques, we need to install the required dependencies:

### Required Libraries:
- **torch** - PyTorch deep learning framework for tensor operations
- **transformers** - Hugging Face library for pre-trained language models
- **accelerate** - Library for distributed training and inference optimization
- **bitsandbytes** - Enables efficient 4-bit quantization to reduce memory usage

### Why These Dependencies?
1. **Memory Efficiency**: 4-bit quantization allows us to run large models on consumer hardware
2. **Model Access**: Transformers library provides easy access to state-of-the-art models
3. **Performance**: Accelerate optimizes inference speed and memory usage

In [ ]:
# Install core dependencies for LLM inference
# torch: PyTorch framework for deep learning operations
# transformers: Hugging Face library for accessing pre-trained models
# accelerate: Optimizes model loading and inference across devices
# bitsandbytes: Enables 4-bit quantization for memory-efficient inference
!pip install torch transformers accelerate bitsandbytes

In [ ]:
# Update bitsandbytes to the latest version for optimal 4-bit quantization support
# The -U flag ensures we get the most recent version with latest optimizations
!pip install -U bitsandbytes

## Model Setup and Prompt Engineering Implementation

### Model Choice: Microsoft Phi-3-mini-4k-instruct
We're using **Phi-3-mini-4k-instruct** because:
- **Compact Size**: ~3.8B parameters, suitable for consumer hardware
- **Instruction-Tuned**: Optimized for following complex prompts and instructions
- **4K Context**: Can handle moderately long conversations and contexts
- **High Quality**: Competitive performance despite smaller size

### Key Implementation Features:

#### **Model Configuration**
- **4-bit Quantization**: Reduces memory usage by ~75% while maintaining quality
- **Float16 Precision**: Balances speed and accuracy
- **Auto Device Mapping**: Automatically distributes model across available GPUs/CPU

#### **Inference Function**
- **Configurable Generation**: Temperature control for creativity vs consistency
- **Token Limits**: Prevents runaway generation with max_new_tokens
- **Clean Output**: Removes special tokens for readable responses

#### **Prompt Engineering Techniques**
Each technique demonstrates a different approach to guide model behavior:

1. **Role Assignment** - Assigns expertise and target audience
2. **Chain-of-Thought** - Encourages step-by-step reasoning
3. **Output Constraints** - Controls format and length precisely
4. **Hypothetical Scenarios** - Provides creative context
5. **Few-Shot Learning** - Shows examples to guide behavior

In [ ]:
# Import required libraries for model loading and inference
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# ===== Model Configuration =====
# Specify the pre-trained model to use for our experiments
# Phi-3-mini-4k-instruct is chosen for its balance of performance and efficiency
model_name = "microsoft/Phi-3-mini-4k-instruct"

print(f"Loading {model_name} in 4-bit mode...")

# Initialize the tokenizer to convert text to tokens and vice versa
# The tokenizer handles text preprocessing and post-processing
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the model with optimized configuration for memory efficiency
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",          # Automatically distribute model across available devices
    torch_dtype=torch.float16,  # Use half-precision to reduce memory usage
    load_in_4bit=True          # Enable 4-bit quantization for maximum memory efficiency
)

def run_inference(prompt):
    """
    Generate LLM response from a given prompt.

    Args:
        prompt (str): The input text prompt to generate a response for

    Returns:
        str: The generated response text (decoded and cleaned)

    This function encapsulates the complete inference pipeline:
    1. Tokenize the input prompt
    2. Generate tokens using the model
    3. Decode tokens back to human-readable text
    """
    # Convert the prompt text into tokens that the model can process
    # return_tensors="pt" ensures PyTorch tensor format
    # .to(model.device) moves tensors to the same device as the model
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate new tokens based on the input prompt
    outputs = model.generate(
        **inputs,                    # Unpack input tensors (input_ids, attention_mask, etc.)
        max_new_tokens=200,         # Limit response length to prevent runaway generation
        temperature=0.7             # Control randomness: 0.0=deterministic, 1.0=very creative
    )

    # Convert generated tokens back to human-readable text
    # skip_special_tokens=True removes technical tokens like <pad>, <eos>, etc.
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# ===== Prompt Engineering Techniques =====
# Dictionary containing 5 different prompting strategies with example prompts
# Each technique demonstrates a different way to guide model behavior
techniques = {
    # 1. ROLE ASSIGNMENT: Assign specific expertise and target audience
    # This technique makes the AI adopt a particular persona or professional role
    "1. Role Assignment":
        "You are a professional science teacher. Explain quantum computing to a 10-year-old in 3 short bullet points.",

    # 2. CHAIN-OF-THOUGHT: Encourage step-by-step reasoning
    # This helps the model break down complex topics into logical steps
    "2. Step-by-Step (Chain-of-Thought)":
        "Explain quantum computing step-by-step, then provide a one-sentence summary at the end.",

    # 3. OUTPUT CONSTRAINTS: Specify exact format and length requirements
    # This gives precise control over response structure and length
    "3. Output Constraints":
        "Explain quantum computing in exactly 4 sentences, each under 12 words.",

    # 4. HYPOTHETICAL SCENARIOS: Provide creative context for explanations
    # This leverages familiar contexts to make complex topics more relatable
    "4. Hypothetical Scenario":
        "Imagine you are a sports commentator explaining quantum computing to football fans.",

    # 5. FEW-SHOT LEARNING: Provide examples to guide the desired output style
    # This shows the model the pattern of responses we want
    "5. Few-Shot Learning":
        "Example: Classical computing is like flipping light switches on or off.\n"
        "Example: Quantum computing is like dimmer switches that can be on, off, or in between.\n"
        "Now give 2 more creative analogies for quantum computing."
}

# ===== Run Experiments =====
# Iterate through each technique and demonstrate its effectiveness
print("Starting Prompt Engineering Experiments...\n")

for name, prompt in techniques.items():
    print(f"\n=== {name} ===")
    print("Prompt:", prompt)
    print("Response:", run_inference(prompt))
    print("-" * 80)  # Visual separator between experiments